In [1]:
!pip install langchain-groq

In [6]:
from langchain_groq import ChatGroq

llm=ChatGroq(model='llama-3.3-70b-versatile',groq_api_key='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'
             ,temperature=0)
response=llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [9]:
!pip install -qU langchain_community beautifulsoup4

In [12]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/analyst-ii-materials-costing/job/R-54158")
page_data=loader.load().pop().page_content
print(page_data)






















Analyst II, Materials Costing










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_extract=PromptTemplate.from_template(
    """
    ### SRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.your job is to extract the job postings and return them in JSON format containing the following keys:'role','experience','skills','description'.
    only return the valid JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)
chain_extract=prompt_extract|llm
res=chain_extract.invoke(input={"page_data":page_data})
print(res.content)
type(res.content)

```json
{
  "role": "Analyst II, Materials Costing",
  "experience": "Minimum +3 years of experience in project management, supply chain operations or related field, with +2 years additional relevant experience in data analytics and design solutions.",
  "skills": [
    "Data analysis tools (e.g., Excel, SQL, ERP systems, Alteryx, Python, Snowflake)",
    "Data visualization tools (e.g., Tableau or PowerBI)",
    "Workflow automation tools (e.g., Smartsheet, Power Automate)",
    "Strong analytical and problem-solving skills"
  ],
  "description": "The ideal candidate will play a key role in optimizing our materials costing management processes, ensuring operational efficiency, and supporting continuous improvement initiatives across the organization. One of your primary responsibilities will be managing the data and analytics for the bi-annual materials negotiations, supporting round-by-round negotiations to optimize allocation of $4B in annual expenditure for cost, performance, susta

str

In [19]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Analyst II, Materials Costing',
 'experience': 'Minimum +3 years of experience in project management, supply chain operations or related field, with +2 years additional relevant experience in data analytics and design solutions.',
 'skills': ['Data analysis tools (e.g., Excel, SQL, ERP systems, Alteryx, Python, Snowflake)',
  'Data visualization tools (e.g., Tableau or PowerBI)',
  'Workflow automation tools (e.g., Smartsheet, Power Automate)',
  'Strong analytical and problem-solving skills'],
 'description': 'The ideal candidate will play a key role in optimizing our materials costing management processes, ensuring operational efficiency, and supporting continuous improvement initiatives across the organization. One of your primary responsibilities will be managing the data and analytics for the bi-annual materials negotiations, supporting round-by-round negotiations to optimize allocation of $4B in annual expenditure for cost, performance, sustainability, and product prior

In [20]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [29]:
import chromadb
import uuid
client=chromadb.PersistentClient("vectorstore")
collection=client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _,row in df.iterrows():
        collection.add(documents=row["Techstack"],metadatas={"links":row["Links"]},ids=[str(uuid.uuid4())])

In [43]:
links=collection.query(query_texts=job['skills'],n_results=2).get('metadatas',[])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}]]

In [38]:
job=json_res
job['skills']

['Data analysis tools (e.g., Excel, SQL, ERP systems, Alteryx, Python, Snowflake)',
 'Data visualization tools (e.g., Tableau or PowerBI)',
 'Workflow automation tools (e.g., Smartsheet, Power Automate)',
 'Strong analytical and problem-solving skills']

In [46]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Anamika, a business development executive at Codemark. Codemark is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Codemark 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Codemark's portfolio: {link_list}
        Remember you are Anamika, BDE at Codemark. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Materials Costing Management and Data Analytics

Dear Hiring Manager,

I came across the job posting for Analyst II, Materials Costing, and I was impressed by the role's focus on optimizing materials costing management processes and supporting continuous improvement initiatives. As a Business Development Executive at Codemark, I believe our company can provide the necessary expertise to support your organization's goals.

At Codemark, we specialize in developing tailored solutions that facilitate the seamless integration of business processes through automated tools. Our team has extensive experience in data analysis, data visualization, and workflow automation, which aligns perfectly with the requirements of the Analyst II, Materials Costing role. We have empowered numerous enterprises with scalable, process-optimized, and cost-reduced solutions, resulting in heightened overall efficiency.

Our expertise in data analysis tools such as Excel, SQL, ERP syst